In [1]:
%matplotlib inline
%pylab inline
import os
from tqdm import tqdm, tqdm_notebook
from tqdm import trange

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pylab as pl

from Bio import SeqIO

from mhcflurry import predict
from mhcflurry import class1_allele_specific

from __future__ import print_function

Populating the interactive namespace from numpy and matplotlib


Using Theano backend.
/home/vts/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
! export KERAS_BACKEND=theano

Loading files:

In [4]:
record_dict = SeqIO.index("data/UP000005640_9606.fasta", "fasta")
# record_dict = SeqIO.to_dict(SeqIO.parse("data/uniprot_sprot.fasta", "fasta"))

In [5]:
list(record_dict.keys())[:9]

['sp|P38646|GRP75_HUMAN',
 'sp|P09958|FURIN_HUMAN',
 'sp|O60383|GDF9_HUMAN',
 'sp|P30511|HLAF_HUMAN',
 'sp|P50995|ANX11_HUMAN',
 'sp|P15880|RS2_HUMAN',
 'sp|Q7RTX7|CTSR4_HUMAN',
 'sp|Q9NPZ5|B3GA2_HUMAN',
 'tr|Q5TEC6|Q5TEC6_HUMAN']

In [6]:
i = 2
len(record_dict['tr|A0A024R161|A0A024R161_HUMAN'].seq[i:i+9])

9

In [15]:
filename = 'output/peptides_raw.txt'
target = open(filename, 'w+')

In [16]:
for key in tqdm(record_dict):
    lines = []
    for i in range(len(record_dict[key].seq)-9):
        lines.append(str(record_dict[key].seq[i:i+9]))
    for i in range(len(lines)):
        target.write(lines[i])
        target.write('\n')

100%|██████████| 21033/21033 [09:11<00:00, 45.85it/s]   


In [17]:
target.close()

In [18]:
df = pd.read_csv('output/peptides_raw.txt', index_col=False)
df.shape

(11314571, 1)

In [19]:
! sort output/peptides_raw.txt | uniq > output/human_peptides.txt

In [2]:
df = pd.read_csv('output/human_peptides.txt', index_col=False)
df.columns = ['Peptide']
df.shape

(10379345, 1)

In [4]:
file1 = 'output/HLA-A0201.csv'
file2 = 'output/human_peptides.txt'
exceptions = 0
with open(file1, 'w') as output:
    output.write('Allele,Peptide,Prediction')
    output.write('\n')
    with open(file2, 'r') as peptides:
        for line in tqdm_notebook(peptides.readlines()):
            s = line.rstrip()
            try:
                prediction = predict(alleles=['HLA-A0201'], peptides=[s])
                output.write('{0},{1},{2}'.format(prediction.Allele.iloc[0], \
                                                prediction.Peptide.iloc[0], prediction.Prediction.iloc[0]))
                output.write('\n')
            except KeyError:
                exceptions += 1
                if (exceptions%10 == 0):
                    print(".", end='')
print('{} exceptions occured'.format(exceptions))


20445/|/  0%|| 20445/10379346 [01:25<11:58:53, 240.16it/s].........................
255 exceptions occured


In [2]:
df = pd.read_csv('output/HLA-A0201.csv', index_col=False)

In [3]:
df.head()

,Allele,Peptide,Prediction
0,HLA-A0201,AAAAAAAAA,3140.859375
1,HLA-A0201,AAAAAAAAC,9386.266602
2,HLA-A0201,AAAAAAAAD,19053.091797
3,HLA-A0201,AAAAAAAAE,13797.026367
4,HLA-A0201,AAAAAAAAF,11891.204102


In [9]:
len(df[df["Prediction"] <= 500])

409088

In [1]:
with open('output/HLA-A0201.csv', 'r') as file:
    with open('output/sample100000.txt', 'w') as file1:
        for i, line in enumerate(file):
            file1.write(line)
            if i == 100000:
                break

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from Levenshtein import hamming
from multiprocessing import Pool
import networkx as nx

In [3]:
data = pd.read_csv('output/HLA-A0201.csv', index_col=False)
data.head()

,Allele,Peptide,Prediction
0,HLA-A0201,AAAAAAAAA,3140.859375
1,HLA-A0201,AAAAAAAAC,9386.266602
2,HLA-A0201,AAAAAAAAD,19053.091797
3,HLA-A0201,AAAAAAAAE,13797.026367
4,HLA-A0201,AAAAAAAAF,11891.204102


In [4]:
binding = 
list2 = []
def ham_dist_dict(data, dist):
    dict_ = dict()
    for i, s_line in enumerate(data['Peptide']):
        result = hamming(line, s_line)
        if result <= dist:
            dict_[i] = result
    return dict_

pool = Pool(processes=4) 

for line in tqdm_notebook(data['Peptide']):
    result = pool.apply_async(ham_dist_dict, (data['Peptide'], 3,))
    list2.append(result)

In [9]:
edges_list = []
for vertex, edges in enumerate(list2):
    for target in edges:
        edges_list.append((vertex, target, edges[target]))

In [12]:
g = nx.Graph()
g.add_weighted_edges_from(edges_list)

In [13]:
del data
del list2

In [ ]:
Gc = max(nx.connected_component_subgraphs(g), key=len)
number_of_nodes(Gc)